## Exercícios

> Retirados de [learn-python: sqlalchemy_orm-questions](https://aviadr1.github.io/learn-advanced-python/11_db_access/exercise/sqlalchemy_orm-questions.html).

#### Q1.

Baixa e extraia o arquivo compactado com o banco de dados [Chinook database](https://www.sqlitetutorial.net/sqlite-sample-database/). Salve o arquivo `chinook.db` na mesma pasta deste script.
* Link para baixar: http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip

<img width=500 src=https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg>


#### Q2.

Leia o código e os comentários das células a seguir para entender como acessamos os modelos ORM de um banco já existente.

In [27]:
from sqlalchemy import create_engine, text, MetaData
from sqlalchemy.orm import Session

engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

### extrai as classes da base de dados Chinook
metadata = MetaData()
metadata.reflect(engine)

# O metadata tem informações sobre as tabelas
# que serão usadas para criar os modelos ORM
for table_name, table in metadata.tables.items():
    print(table_name)
    print(table.columns.keys())
    print(table.columns.items())
    print('-'*25)

### configura o objeto Base mapeando os modelos ORM das tabelas
from sqlalchemy.ext.automap import automap_base
Base = automap_base(metadata=metadata)
Base.prepare()

# o objeto Base tem os modelos ORM que podemos usar
# para manipular o banco de dados
print(Base.classes.items())

albums
['AlbumId', 'Title', 'ArtistId']
[('AlbumId', Column('AlbumId', INTEGER(), table=<albums>, primary_key=True, nullable=False)), ('Title', Column('Title', NVARCHAR(length=160), table=<albums>, nullable=False)), ('ArtistId', Column('ArtistId', INTEGER(), ForeignKey('artists.ArtistId'), table=<albums>, nullable=False))]
-------------------------
artists
['ArtistId', 'Name']
[('ArtistId', Column('ArtistId', INTEGER(), table=<artists>, primary_key=True, nullable=False)), ('Name', Column('Name', NVARCHAR(length=120), table=<artists>))]
-------------------------
customers
['CustomerId', 'FirstName', 'LastName', 'Company', 'Address', 'City', 'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email', 'SupportRepId']
[('CustomerId', Column('CustomerId', INTEGER(), table=<customers>, primary_key=True, nullable=False)), ('FirstName', Column('FirstName', NVARCHAR(length=40), table=<customers>, nullable=False)), ('LastName', Column('LastName', NVARCHAR(length=20), table=<customers>, nullable=F

In [28]:
# A seguir um exemplo de query na tabela Albums
# usamos o objeto Base para acessar cada modelo ORM.

from sqlalchemy import select

session = Session(engine)
res = session.scalars(select(Base.classes.albums))
first_album = res.first()
print(first_album.AlbumId, first_album.Title)

1 For Those About To Rock We Salute You


#### Q3. 
Com base nos códigos anteriores realize as operações solicitadas nas células a seguir:


In [31]:
### Imprima os três primeiros registros da tabela tracks

query = select(Base.classes.tracks).limit(3)

result = session.execute(query).scalars().all()

for row in result:
    session.expunge(row) #To not break the code when it execute a second time

    #parse to dict so i don't have to type every column in the row
    row_dict = row.__dict__
    row_dict.pop('_sa_instance_state', None)
    print(row_dict)

{'Name': 'For Those About To Rock (We Salute You)', 'MediaTypeId': 1, 'Composer': 'Angus Young, Malcolm Young, Brian Johnson', 'Milliseconds': 343719, 'UnitPrice': Decimal('0.99'), 'TrackId': 1, 'AlbumId': 1, 'GenreId': 1, 'Bytes': 11170334}
{'Name': 'Balls to the Wall', 'MediaTypeId': 2, 'Composer': None, 'Milliseconds': 342562, 'UnitPrice': Decimal('0.99'), 'TrackId': 2, 'AlbumId': 2, 'GenreId': 1, 'Bytes': 5510424}
{'Name': 'Fast As a Shark', 'MediaTypeId': 2, 'Composer': 'F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman', 'Milliseconds': 230619, 'UnitPrice': Decimal('0.99'), 'TrackId': 3, 'AlbumId': 3, 'GenreId': 1, 'Bytes': 3990994}


In [39]:
### Imprima o nome da faixa e o título do álbum das primeiras 20 faixas na tabela tracks.
query = (select(
    Base.classes.tracks,
    Base.classes.albums
)   
    .where(Base.classes.albums.AlbumId == Base.classes.tracks.AlbumId)
    .order_by(Base.classes.tracks.TrackId)
    .limit(20)
)

result = session.execute(query).all()

for track, album in result:
    print(f'faixa: {track.Name}, título do álbum: {album.Title}')

faixa: For Those About To Rock (We Salute You), título do álbum: For Those About To Rock We Salute You
faixa: Balls to the Wall, título do álbum: Balls to the Wall
faixa: Fast As a Shark, título do álbum: Restless and Wild
faixa: Restless and Wild, título do álbum: Restless and Wild
faixa: Princess of the Dawn, título do álbum: Restless and Wild
faixa: Put The Finger On You, título do álbum: For Those About To Rock We Salute You
faixa: Let's Get It Up, título do álbum: For Those About To Rock We Salute You
faixa: Inject The Venom, título do álbum: For Those About To Rock We Salute You
faixa: Snowballed, título do álbum: For Those About To Rock We Salute You
faixa: Evil Walks, título do álbum: For Those About To Rock We Salute You
faixa: C.O.D., título do álbum: For Those About To Rock We Salute You
faixa: Breaking The Rules, título do álbum: For Those About To Rock We Salute You
faixa: Night Of The Long Knives, título do álbum: For Those About To Rock We Salute You
faixa: Spellbound, t

In [41]:
### Imprima as 10 primeiras vendas de faixas da tabela invoice_items
### Para essas 10 primeiras vendas, imprima os nomes das faixas vendidas e a quantidade vendida.

query = (select(
    Base.classes.invoice_items,
    Base.classes.tracks
)
.where(Base.classes.tracks.TrackId == Base.classes.invoice_items.TrackId)
.order_by(Base.classes.invoice_items.InvoiceLineId)
.limit(10)
)

result = session.execute(query).all()

for item, track in result:
    print(f'Nome: {track.Name}, quantidade vendida: {item.Quantity}')

Nome: Balls to the Wall, quantidade vendida: 1
Nome: Restless and Wild, quantidade vendida: 1
Nome: Put The Finger On You, quantidade vendida: 1
Nome: Inject The Venom, quantidade vendida: 1
Nome: Evil Walks, quantidade vendida: 1
Nome: Breaking The Rules, quantidade vendida: 1
Nome: Dog Eat Dog, quantidade vendida: 1
Nome: Overdose, quantidade vendida: 1
Nome: Love In An Elevator, quantidade vendida: 1
Nome: Janie's Got A Gun, quantidade vendida: 1


In [47]:
### Imprima os nomes das 10 faixas mais vendidas e quantas vezes foram vendidas.

from sqlalchemy import func,desc

query = (select(
    Base.classes.tracks.Name,
    func.sum(Base.classes.invoice_items.Quantity).label('total_sales')
)
.where(Base.classes.invoice_items.TrackId == Base.classes.tracks.TrackId)
.group_by(Base.classes.tracks.Name)
.order_by(desc('total_sales'))
.limit(10)
)

result = session.execute(query).all()

for name, sales in result:
    print(f'Nome: {name}, quantidade vendida: {sales}')

Nome: The Trooper, quantidade vendida: 5
Nome: Untitled, quantidade vendida: 4
Nome: The Number Of The Beast, quantidade vendida: 4
Nome: Sure Know Something, quantidade vendida: 4
Nome: Hallowed Be Thy Name, quantidade vendida: 4
Nome: Eruption, quantidade vendida: 4
Nome: Where Eagles Dare, quantidade vendida: 3
Nome: Welcome Home (Sanitarium), quantidade vendida: 3
Nome: Sweetest Thing, quantidade vendida: 3
Nome: Surrender, quantidade vendida: 3


In [48]:
### Quem são os 10 artistas que mais venderam?
### dica: você precisa juntar as tabelas invoice_items, tracks, albums e artists

invoice_items = Base.classes.invoice_items
tracks = Base.classes.tracks
albums = Base. classes.albums
artists = Base.classes.artists

from sqlalchemy import func,desc

query = (select(
    artists.Name,
    func.sum(invoice_items.Quantity).label('total_sales')
)
.join(tracks, invoice_items.TrackId == tracks.TrackId)
.join(albums, tracks.AlbumId == albums.AlbumId)
.join(artists, albums.ArtistId == artists.ArtistId)
.group_by(artists.Name)
.order_by(desc('total_sales'))
.limit(10)
)

result = session.execute(query).all()

for name, sales in result:
    print(f'Nome: {name}, quantidade vendida: {sales}')


Nome: Iron Maiden, quantidade vendida: 140
Nome: U2, quantidade vendida: 107
Nome: Metallica, quantidade vendida: 91
Nome: Led Zeppelin, quantidade vendida: 87
Nome: Os Paralamas Do Sucesso, quantidade vendida: 45
Nome: Deep Purple, quantidade vendida: 44
Nome: Faith No More, quantidade vendida: 42
Nome: Lost, quantidade vendida: 41
Nome: Eric Clapton, quantidade vendida: 40
Nome: R.E.M., quantidade vendida: 39
